In [10]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import sys

In [11]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sample_with_prob(arr): 
  random = np.random.uniform(-1,1, arr.shape)

  array = np.ones(arr.shape)
  array[np.where(arr<random)] = -1
  return array
    # arr = (arr + 1) / 2 # from tanh to probability

    # return np.array([np.random.choice([-1,1], p=[1-prob,prob]) for prob in arr])

# Convolution
def convolution(W,Prev,stride):
  Prev_cp = np.concatenate((Prev.copy(), Prev.copy()), axis=0) # extend Prev array
  res = []
  for i in range(0, len(Prev), stride):
    res.append( W @ Prev_cp[i:i+2])
  return np.array(res)

# Energy Function
def energy_function(W,h,v,stride,b,c):
    return np.multiply(-h.T, convolution(W,v,stride)).sum() - b*sum(h) - c*sum(v)

In [12]:
Case = 1
# I) 1D Convolutional RBM properties
if Case == 1:
    L = 10
    stride = 1
    W = [2.346, -2.346] # filter
    bias_hidden = 0
    bias_visible = 0
elif Case == 2:
    L = 10
    stride = 1
    W = [2.346, 2.346] # filter
    bias_hidden = 0
    bias_visible = 0

V = np.array([1 for _ in range(L)])
H = np.array([0 for _ in range(L)])

num_samples = 500
samples = []
min_energy = float('inf')
min_energy_sample = V.copy()


In [13]:
for it in range(num_samples*2):
    if it%2 == 0: # V to H
        H_conv = convolution(W,V,stride)
        tanh_vec = np.array(np.tanh(H_conv))
        H = np.array(sample_with_prob(tanh_vec))
    else: # H to V
        # print(np.flip(W), np.array(W).T)
        V_conv = convolution(np.flip(W),H,stride)
        tanh_vec = np.array(np.tanh(np.roll(V_conv,1)))
        V = np.array(sample_with_prob(tanh_vec))
        istr = [str(i) for i in V]
        samples.append(",".join(istr))
        energy = energy_function(W,H,V,stride,bias_visible,bias_hidden)
        if energy < min_energy:
            min_energy_sample = V.copy()
            min_energy = energy


In [14]:
value_counts = pd.value_counts(samples)
print("Max Occurance: ", samples[value_counts.argmax()], ": ",
        value_counts[value_counts.argmax()], "times out of ", num_samples, "times")
print("Min Energy : ", min_energy, " is at ", min_energy_sample)

#print(value_counts)
#plt.hist(samples)
#plt.show()

Max Occurance:  -1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0 :  493 times out of  500 times
Min Energy :  -46.92  is at  [-1.  1. -1.  1. -1.  1. -1.  1. -1.  1.]
